In [ ]:
# ipo_date before want_date to csv

import numpy as np
import datetime
import pandas as pd
kosdaq_data = pd.read_csv('KOSDAQ.csv')
kosdaq_data['ListingDate'] = pd.to_datetime(kosdaq_data['ListingDate'], format='%Y-%m-%d')
kosdaq_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
want_date = np.datetime64(datetime.date(2016,1,1))
anal_kosdaq_data = kosdaq_data[kosdaq_data['ListingDate'] < want_date]
anal_kosdaq_data.reset_index(drop=True, inplace = True)
anal_kosdaq_data.to_csv('A_KOSDAQ.csv')

In [ ]:
#get firm_code on google_trends
import numpy as np
import datetime
import pandas as pd
import pytrends
import time
from pytrends.request import TrendReq
from pytrends.exceptions import ResponseError
pytrends = TrendReq()



def _fetch_data_suggestions(trendreq, keyword : str) -> pd.DataFrame:

    """Download google trends data using pytrends TrendReq and retries in case of a ResponseError."""
    attempts, fetched = 0, False
    while not fetched:
        try:
            keyword_codes = pytrends.suggestions(keyword)
        except ResponseError as err:
            print(err)
            print(f'Trying again in {60 + 5 * attempts} seconds.')
            time.sleep(60 + 5 * attempts)
            attempts += 1
            if attempts > 3:
                print('Failed after 3 attemps, abort fetching.')
                break
        else:
            fetched = True
    return keyword_codes

def _fetch_data(trendreq, kw_list, timeframe='today 5-y', cat=0, geo='', gprop='') -> pd.DataFrame:
    
    """Download google trends data using pytrends TrendReq and retries in case of a ResponseError."""
    attempts, fetched = 0, False
    while not fetched:
        try:
            trendreq.build_payload(kw_list=kw_list, timeframe=timeframe, cat=cat, geo=geo, gprop=gprop)
        except ResponseError as err:
            print(err)
            print(f'Trying again in {60 + 5 * attempts} seconds.')
            time.sleep(60 + 5 * attempts)
            attempts += 1
            if attempts > 3:
                print('Failed after 3 attemps, abort fetching.')
                break
        else:
            fetched = True
    return trendreq.interest_over_time()

def _fetch_data_region(trendreq, kw_list, timeframe='today 5-y', cat=0, geo='', gprop='') -> pd.DataFrame:
    
    """Download google trends data using pytrends TrendReq and retries in case of a ResponseError."""
    attempts, fetched = 0, False
    while not fetched:
        try:
            trendreq.build_payload(kw_list=kw_list, timeframe=timeframe, cat=cat, geo=geo, gprop=gprop)
        except ResponseError as err:
            print(err)
            print(f'Trying again in {60 + 5 * attempts} seconds.')
            time.sleep(60 + 5 * attempts)
            attempts += 1
            if attempts > 3:
                print('Failed after 3 attemps, abort fetching.')
                break
        else:
            fetched = True
    return trendreq.interest_by_region()

df = pd.read_csv('A_KOSDAQ.csv')
KEYWORDS = df['Name'].tolist()
KEYWORDS_NAMES_CODES = []

for i in KEYWORDS:
    KEYWORDS_CODES = _fetch_data_suggestions(pytrends, keyword = i)
    if not KEYWORDS_CODES:
        continue
    df_CODES = pd.DataFrame(KEYWORDS_CODES)
    for r in range(df_CODES.index.size):
        if 'company' in df_CODES.loc[r, 'type'].lower():
            company_CODE = df_CODES.loc[r, 'mid']
            df_region = _fetch_data_region(pytrends, kw_list = [company_CODE])
            if df_region.loc['South Korea', company_CODE] > 80:
                #consider this as firm_analyse code
                KEYWORDS_NAMES_CODES.append([i, company_CODE])
                break
        else:
            df_region = _fetch_data_region(pytrends, kw_list = [i])
            if df_region.loc['South Korea', i] > 80:
                KEYWORDS_NAMES_CODES.append([i, i])
                break

left = pd.read_csv('A_KOSDAQ.csv')
right = pd.DataFrame(np.array(KEYWORDS_NAMES_CODES),columns=['Name', 'Firm_Code'])
AN_KOSDAQ = left.merge(right, how='left', on='Name').drop(['Unnamed: 0'], axis = 1)
AN_KOSDAQ.to_csv('AN_KOSDAQ.csv')

In [ ]:
#erase row that has nan
import pandas as pd

df = pd.read_csv('AN_KOSDAQ.csv')
df.dropna(subset = ['Firm_Code'], inplace = True)
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df.reset_index(drop=True, inplace = True)
df.to_csv('ANA_KOSDAQ.csv')

In [ ]:
#get google_trends data of keywords
import numpy as np
import datetime
import pandas as pd
import pytrends
import time
from pytrends.request import TrendReq
from pytrends.exceptions import ResponseError
pytrends = TrendReq()

def _fetch_data(trendreq, kw_list, timeframe='today 5-y', cat=0, geo='', gprop='') -> pd.DataFrame:
    
    """Download google trends data using pytrends TrendReq and retries in case of a ResponseError."""
    attempts, fetched = 0, False
    while not fetched:
        try:
            trendreq.build_payload(kw_list=kw_list, timeframe=timeframe, cat=cat, geo=geo, gprop=gprop)
        except ResponseError as err:
            print(err)
            print(f'Trying again in {60 + 5 * attempts} seconds.')
            time.sleep(60 + 5 * attempts)
            attempts += 1
            if attempts > 3:
                print('Failed after 3 attemps, abort fetching.')
                break
        else:
            fetched = True
    return trendreq.interest_over_time()

df_KOSDAQ = pd.read_csv('ANA_KOSDAQ.csv')
KEYWORDS_CODES = df_KOSDAQ['Firm_Code'].to_list()

Individual_EXACT_KEYWORD = list(zip(*[iter(KEYWORDS_CODES)]*1))
Individual_EXACT_KEYWORD = [list(x) for x in Individual_EXACT_KEYWORD]
dicti = {}
i = 1
for keyword in Individual_EXACT_KEYWORD:
    keyword_trend = _fetch_data(pytrends, keyword)
    if keyword_trend is not None:
        dicti[i] = keyword_trend
        i+=1
    else:
        continue
df_trends = pd.concat(dicti, axis=1)
#make df_trends into dataframe
df_trends.columns = df_trends.columns.droplevel(0) #drop outside header
df_trends = df_trends.drop('isPartial', axis = 1) #drop "isPartial"
df_trends.reset_index(level=0,inplace=True) #reset_index
column_names = ['date']
column_names.extend(KEYWORDS_CODES)
df_trends.columns = column_names #change column names
df_trends.to_csv('KOSDAQ_TREND.csv') #make into csv    

In [ ]:
#erase the column that has no name
import pandas as pd

df = pd.read_csv('KOSDAQ_TREND.csv')
df.drop(df.filter(regex="Unname"),axis=1, inplace=True)
df.rename(columns={"data": "date"}, inplace = True)
df.to_csv('KOSDAQ_TREND.csv')

In [ ]:
#get data when data_boom
import pandas as pd
import pytrends
import numpy as np
from datetime import date
import datetime as dt
import time
from dateutil.relativedelta import relativedelta
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

def outlier_treatment(datacolumn):
    sorted(datacolumn)
    Q1,Q3 = np.percentile(datacolumn , [25,75])
    IQR = Q3 - Q1
    upper_range = Q3 + (2.0* IQR)
    return upper_range

trend_KOSDAQ = pd.read_csv('KOSDAQ_TREND.csv')
#trend_KOSDAQ['date'] = pd.to_datetime(trend_KOSDAQ['date'], format='%Y-%m-%d')
df_KOSDAQ = pd.read_csv('ANA_KOSDAQ.csv')
KEYWORDS_CODES = df_KOSDAQ['Firm_Code'].to_list()
KEYWORDS_BOOM_DATES = []

for keyword in KEYWORDS_CODES:
        #이거 그냥 바로 column 까지 나오는지 확인
    keyword_trend = trend_KOSDAQ[['date', keyword]]
    upperbound = outlier_treatment(keyword_trend[keyword])
    if upperbound != 0:
        n_keyword_trend = keyword_trend.copy()
        n_keyword_trend.drop(n_keyword_trend[ (n_keyword_trend[keyword] > upperbound) ].index , inplace=True)
        n_upperbound = outlier_treatment(n_keyword_trend[keyword])
        n_upper_outlier = keyword_trend[keyword_trend[keyword] > n_upperbound]
        n_upper_outlier.reset_index(drop = True, inplace = True)

        old_date = date(2014, 1, 1)
        if n_upper_outlier.index.size != 0:
            for i in range(n_upper_outlier.index.size):
                new_date = n_upper_outlier.loc[i, 'date']
                new_date = dt.datetime.strptime(new_date, "%Y-%m-%d")
                new_date = new_date.date()
                if new_date > old_date + relativedelta(years=1):
                    KEYWORDS_BOOM_DATES.append([keyword, new_date, round(n_upper_outlier.loc[i, keyword]/upperbound,2)])
                    old_date = new_date
                else:
                    continue
        else:
            continue
    else:
        continue
                
left = pd.read_csv('ANA_KOSDAQ.csv')
right = pd.DataFrame(np.array(KEYWORDS_BOOM_DATES), columns=['Firm_Code', 'Boom_Date', 'How_Big'])
AN_KOSDAQ = left.merge(right, how='inner', on='Firm_Code').drop(['Unnamed: 0'], axis = 1)
AN_KOSDAQ.to_csv('ANAL_KOSDAQ.csv')      

In [ ]:
#get data of only name and boom date and how big
import pandas as pd
df = pd.read_csv('ANAL_KOSDAQ.csv')
trend = df[['Name', 'Boom_Date', 'How_Big']]
print(df['How_Big'].dtypes)
n_upperbound = 2.0
more_trend = trend[trend.How_Big > n_upperbound]
more_trend.to_csv('more_trend.csv')